In [ ]:
!pip install gensim

In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split

import gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from keras.utils import to_categorical

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\e_enr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
music = pd.read_csv('Dataset/FullDataSet/Train.csv')
music.head()

,index,title,artist,words,category,lyrics
0,1,All Messed Up,Sum 41,164,angry,Another day wasted out of time I can't get out...
1,2,All Nightmare Long,Metallica,477,angry,Crawl from the wreckage one more time Horrific...
2,3,American Idiot (Album Version),Green Day,176,angry,Don't wanna be an American idiot. Don't want ...
3,4,Angry Again,Megadeth,242,angry,The more of you that I inspect The more of me ...
4,5,Annihilation By The Hands Of God (Album Version),Roadrunner United,333,angry,The prophets of God have left you solaced inde...


In [3]:
music.shape

(400, 6)

In [4]:
music["category"].value_counts()

relaxed    100
happy      100
sad        100
angry      100
Name: category, dtype: int64

In [5]:
# Consolidate sad and relaxed
mapping = {'happy': 1, 'sad': 0, 'relaxed': 0, 'angry': -1}
new_music = music.replace({'category': mapping})
new_music

,index,title,artist,words,category,lyrics
0,1,All Messed Up,Sum 41,164,-1,Another day wasted out of time I can't get out...
1,2,All Nightmare Long,Metallica,477,-1,Crawl from the wreckage one more time Horrific...
2,3,American Idiot (Album Version),Green Day,176,-1,Don't wanna be an American idiot. Don't want ...
3,4,Angry Again,Megadeth,242,-1,The more of you that I inspect The more of me ...
4,5,Annihilation By The Hands Of God (Album Version),Roadrunner United,333,-1,The prophets of God have left you solaced inde...
5,6,Another Time Around,Sum 41,416,-1,"In my years, I've seen all sounds of misconcep..."
6,7,Awaken,Disturbed,269,-1,"Stripped of life, alone In the midst of someth..."
7,8,Ballad Of Ira Hayes,Johnny Cash,173,-1,Ira Hayes Ira Hayes Call him drunken Ira Haye...
8,9,Battery,Metallica,316,-1,"Lashing out the action, returning the reaction..."
9,10,Before You Die,Bad Religion,412,-1,Decayingâ ever faster. Under wear all the im...


In [6]:
new_music = music.drop("index", axis=1)

In [7]:
stopwords = stopwords.words('english')

In [8]:
documents = [text.split() for text in music.lyrics]

In [9]:
len(documents)

400

In [10]:
w2v_model = gensim.models.word2vec.Word2Vec(size = 256, window = 7, min_count = 5)

In [11]:
w2v_model.build_vocab(documents)

In [12]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(2359687, 3555392)

In [13]:
w2v_model.wv["run"]

array([ 0.6791371 ,  0.44889694, -0.4138222 ,  0.4370413 ,  0.63298625,
        0.18099274, -0.5414545 ,  0.21693884,  0.06580668, -0.26714194,
        0.8790139 , -0.45574695,  0.18546948, -0.10576976, -0.11590587,
        1.0129281 ,  0.16674192, -0.06108264, -0.04817478,  0.378605  ,
       -0.2609911 ,  0.03317019,  0.3266772 ,  0.46849442, -0.1511256 ,
        0.11337073,  0.74304134, -0.36291558, -0.8580551 ,  0.283863  ,
        0.23021646, -0.83422387, -0.4822133 ,  0.10766821, -0.19175537,
       -0.5324786 ,  0.09143832,  0.12757406, -0.84080905, -0.2893894 ,
       -0.60803676,  0.8766407 ,  1.6581597 , -0.37375337, -0.4219403 ,
        0.06566507,  0.05878248,  0.09944476,  0.8235737 , -0.21738672,
        0.5291103 ,  0.32759365,  0.3803465 ,  1.1061226 , -0.26310033,
        1.0091014 ,  0.19100027,  0.4835705 ,  0.05413153, -0.36487812,
       -0.13914031,  1.4943509 , -0.10625819, -0.16539527,  0.17776282,
       -0.19608444, -0.70014197, -0.98176455,  0.8338984 ,  0.03

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(music.lyrics)

In [15]:
x_train = pad_sequences(tokenizer.texts_to_sequences(music.lyrics), maxlen=256, padding="post", truncating="post")

In [16]:
x_train

array([[ 182,   88, 1271, ...,    0,    0,    0],
       [ 846,   59,    3, ...,    0,    0,    0],
       [  26,   70,   17, ...,    0,    0,    0],
       ...,
       [  32,  795,   12, ...,    0,    0,    0],
       [  22,   16,   29, ...,    0,    0,    0],
       [1270,  580,   50, ..., 6471, 1237,   29]])

In [17]:
mapping = {'happy': 1, 'sad': 0, 'relaxed': 0, 'angry': -1}
new_music = music.replace({'category': mapping})
y_train = new_music.category

In [18]:
y_train

0     -1
1     -1
2     -1
3     -1
4     -1
5     -1
6     -1
7     -1
8     -1
9     -1
10    -1
11    -1
12    -1
13    -1
14    -1
15    -1
16    -1
17    -1
18    -1
19    -1
20    -1
21    -1
22    -1
23    -1
24    -1
25    -1
26    -1
27    -1
28    -1
29    -1
      ..
370    0
371    0
372    0
373    0
374    0
375    0
376    0
377    0
378    0
379    0
380    0
381    0
382    0
383    0
384    0
385    0
386    0
387    0
388    0
389    0
390    0
391    0
392    0
393    0
394    0
395    0
396    0
397    0
398    0
399    0
Name: category, Length: 400, dtype: int64

In [19]:
y_train_f = []
for x in y_train:
    if x == 1:
        y_train_f.append(1)
    elif x == 0:
        y_train_f.append(0)
    elif x == -1:
        y_train_f.append(2)
        
y_train_f = to_categorical(y_train_f)

In [20]:
y_train_f

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [21]:
embedding_matrix = np.zeros((14850,256))

In [22]:
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [23]:
embedding_layer = Embedding(14850, 256, weights=[embedding_matrix], input_length=256, trainable=False)

In [24]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.25))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(50, activation="relu"))
model.add(Dense(3, activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 256)          3801600   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 256)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 256, 200)          51400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_2 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 153       
Total para

In [26]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [27]:
len(x_train)

400

In [28]:
len(y_train_f)

400

In [30]:
model.fit(x_train, y_train_f, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Train on 360 samples, validate on 40 samples
Epoch 1/10
360/360 [==============================] - 6s 16ms/step - loss: 0.9803 - acc: 0.5583 - val_loss: 0.5483 - val_acc: 0.9500
Epoch 2/10
360/360 [==============================] - 6s 16ms/step - loss: 0.9598 - acc: 0.5500 - val_loss: 1.0096 - val_acc: 0.7000
Epoch 3/10
360/360 [==============================] - 6s 16ms/step - loss: 0.9187 - acc: 0.6111 - val_loss: 0.8196 - val_acc: 0.8000
Epoch 4/10
360/360 [==============================] - 6s 16ms/step - loss: 0.9115 - acc: 0.5833 - val_loss: 0.7066 - val_acc: 0.8000
Epoch 5/10
360/360 [==============================] - 6s 16ms/step - loss: 0.8617 - acc: 0.6194 - val_loss: 0.8369 - val_acc: 0.7750
Epoch 6/10
360/360 [==============================] - 6s 16ms/step - loss: 0.8437 - acc: 0.6139 - val_loss: 1.0286 - val_acc: 0.7250
Epoch 7/10
360/360 [==============================] - 6s 16ms/step - loss: 0.8341 - acc: 0.6167 - val_loss: 0.9418 - val_acc: 0.7500
Epoch 8/10
360/360 [====

In [31]:
model_loss, model_accuracy = model.evaluate(
    x_train, y_train_f, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.6446817111968994, Accuracy: 0.7575


In [32]:
model.save("music_lyric_analyzer.h5")